In [5]:
import pandas as pd
import re
from tqdm import tqdm

In [6]:
kospi = pd.read_csv("KOSPI200.csv")
kospi.loc[len(kospi)] = [kospi.columns[0], kospi.columns[1]]
kospi.columns = ["code", "name"]
kospi

,code,name
0,373220,LG에너지솔루션
1,660,SK하이닉스
2,207940,삼성바이오로직스
3,6400,삼성SDI
4,51910,LG화학
...,...,...
195,57050,현대홈쇼핑
196,284740,쿠쿠홈시스
197,13890,지누스
198,241590,화승엔터프라이즈


In [ ]:
import dart_fss as dart

# Open DART API KEY 설정
api_key='21fddf42c2c82c8c7587e0e74147f05d048da5e4'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

# 삼성전자 검색
for i in tqdm(range(len(kospi))):
    name = kospi.iloc[i]["name"]
    try:
        samsung = corp_list.find_by_corp_name(name, exactly=True)[0]
        fs = samsung.extract_fs(bgn_de='19500101')
        fs.save(name+".xlsx")
        data = pd.read_excel("fsdata/"+name+".xlsx", sheet_name="Data_cf", index_col=0)
        col = list(data.columns)
        date = []
        for i in col:
            regex = r'\d{4}\d{2}\d{2}-\d{4}\d{2}\d{2}'
            if bool(re.match(regex, i)):
                date.append(i)
        col_name = list(data["Unnamed: 2"].iloc[2:])
        clean_data = []
        for i in date:
            value = list(data[i].iloc[2:])
            clean_data.append(value)
        date = [i[:8] for i in date]
        clean_data = pd.DataFrame(clean_data, columns = col_name, index=date)
        clean_data = clean_data.sort_index(ascending=True)
        clean_data.to_csv("Data/Statements_data/"+name+".csv")
    except:
        print("None Data :",name)